# Making Your Model Learn Addition!
## Task 1: Introduction

Given the string "54+7", the model should return a prediction: "61".

In [21]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

## Task 2: Generate Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [22]:
all_chars = '0123456789+'

In [23]:
num_features = len(all_chars)

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))

print('Number of features:', num_features)

Number of features: 11


In [24]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    example = str(first_num) + '+' + str(second_num)
    label = str(first_num+second_num)
    return example, label

generate_data()

('29+17', '46')

## Task 3: Create the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___
Consider these two reviews:

Review 1: This movie is not terrible at all.

Review 2: This movie is pretty decent.

In [25]:
hidden_units = 128
max_time_steps = 5

model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


## Task 4: Vectorize and De-Vectorize Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [26]:
def vectorize_example(example, label):
    
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 30+41 71
Vectorized Example and Label Shapes: (5, 11) (5, 11)


In [27]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

devectorize_example(x)

'30+41'

In [28]:
devectorize_example(y)

'00071'

## Task 5: Create Dataset
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [29]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, max_time_steps, num_features))
    y_train = np.zeros((num_examples, max_time_steps, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset()
print(x_train.shape, y_train.shape)

(2000, 5, 11) (2000, 5, 11)


In [30]:
devectorize_example(x_train[0])

'71+83'

In [31]:
devectorize_example(y_train[0])

'00154'

## Task 6: Training the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [33]:
l_cb = LambdaCallback(
    on_epoch_end = lambda e, l: print('{:.2f}'.format(l['val_acc']), end = ' _ ')
)
es_cb = EarlyStopping(monitor = 'val_loss', patience = 10)
model.fit(x_train,y_train, epochs = 500, batch_size = 256, validation_split = 0.2,
         verbose = False, callbacks = [es_cb, l_cb])

0.55 _ 0.61 _ 0.61 _ 0.61 _ 0.62 _ 0.62 _ 0.62 _ 0.64 _ 0.64 _ 0.65 _ 0.65 _ 0.64 _ 0.65 _ 0.65 _ 0.65 _ 0.66 _ 0.66 _ 0.67 _ 0.66 _ 0.67 _ 0.67 _ 0.67 _ 0.68 _ 0.69 _ 0.69 _ 0.68 _ 0.70 _ 0.70 _ 0.70 _ 0.71 _ 0.70 _ 0.71 _ 0.70 _ 0.71 _ 0.72 _ 0.72 _ 0.72 _ 0.73 _ 0.74 _ 0.74 _ 0.74 _ 0.75 _ 0.74 _ 0.75 _ 0.75 _ 0.75 _ 0.76 _ 0.76 _ 0.76 _ 0.76 _ 0.77 _ 0.77 _ 0.77 _ 0.78 _ 0.78 _ 0.78 _ 0.78 _ 0.78 _ 0.79 _ 0.79 _ 0.79 _ 0.79 _ 0.80 _ 0.80 _ 0.80 _ 0.81 _ 0.80 _ 0.81 _ 0.82 _ 0.81 _ 0.82 _ 0.82 _ 0.82 _ 0.84 _ 0.83 _ 0.83 _ 0.84 _ 0.84 _ 0.84 _ 0.85 _ 0.85 _ 0.85 _ 0.85 _ 0.85 _ 0.85 _ 0.86 _ 0.86 _ 0.87 _ 0.87 _ 0.87 _ 0.87 _ 0.87 _ 0.87 _ 0.88 _ 0.88 _ 0.89 _ 0.89 _ 0.89 _ 0.89 _ 0.89 _ 0.89 _ 0.90 _ 0.90 _ 0.90 _ 0.90 _ 0.90 _ 0.90 _ 0.91 _ 0.91 _ 0.90 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.92 _ 0.93 _ 0.94 _ 0.93 _ 0.93 _ 0.94 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.94 _

In [49]:
x_test, y_test = create_dataset(10)
preds = model.predict(x_test)
for i, pred in enumerate(preds):
    y = devectorize_example(y_test[i])
    y_hat = devectorize_example(pred)
    col = 'blue'
    if y!= y_hat:
        col = 'red'
    out = 'Input: '+devectorize_example(x_test[i]) + ' Out: '+y+ ' Pred: ' +y_hat
    print(colored(out,col))

Input: 05+62 Out: 00067 Pred: 00067
Input: 92+89 Out: 00181 Pred: 00181
Input: 08+80 Out: 00088 Pred: 00088
Input: 20+76 Out: 00096 Pred: 00096
Input: 24+69 Out: 00093 Pred: 00092
Input: 62+53 Out: 00115 Pred: 00115
Input: 91+97 Out: 00188 Pred: 00188
Input: 23+72 Out: 00095 Pred: 00095
Input: 074+0 Out: 00074 Pred: 00075
Input: 43+93 Out: 00136 Pred: 00136
